In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the images to range [-1, 1]
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)


100%|██████████| 170498071/170498071 [00:13<00:00, 12904495.36it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
resnet_model = models.resnet18(pretrained=False)
resnet_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet_model.parameters(), lr=0.001, momentum=0.9)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = resnet_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # Print every 200 mini-batches
            print(f"[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 200:.3f}")
            running_loss = 0.0

print("Finished Training ResNet18 on CIFAR-10")

[Epoch 1, Batch 200] Loss: 2.263
[Epoch 1, Batch 400] Loss: 1.596
[Epoch 1, Batch 600] Loss: 1.479
[Epoch 2, Batch 200] Loss: 1.280
[Epoch 2, Batch 400] Loss: 1.251
[Epoch 2, Batch 600] Loss: 1.219
[Epoch 3, Batch 200] Loss: 1.051
[Epoch 3, Batch 400] Loss: 1.022
[Epoch 3, Batch 600] Loss: 1.036
[Epoch 4, Batch 200] Loss: 0.868
[Epoch 4, Batch 400] Loss: 0.857
[Epoch 4, Batch 600] Loss: 0.884
[Epoch 5, Batch 200] Loss: 0.704
[Epoch 5, Batch 400] Loss: 0.731
[Epoch 5, Batch 600] Loss: 0.766
[Epoch 6, Batch 200] Loss: 0.561
[Epoch 6, Batch 400] Loss: 0.613
[Epoch 6, Batch 600] Loss: 0.625
[Epoch 7, Batch 200] Loss: 0.453
[Epoch 7, Batch 400] Loss: 0.483
[Epoch 7, Batch 600] Loss: 0.518
[Epoch 8, Batch 200] Loss: 0.358
[Epoch 8, Batch 400] Loss: 0.388
[Epoch 8, Batch 600] Loss: 0.409
[Epoch 9, Batch 200] Loss: 0.284
[Epoch 9, Batch 400] Loss: 0.302
[Epoch 9, Batch 600] Loss: 0.360
[Epoch 10, Batch 200] Loss: 0.223
[Epoch 10, Batch 400] Loss: 0.234
[Epoch 10, Batch 600] Loss: 0.273
Finishe

In [6]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True, num_workers=2)

Files already downloaded and verified


In [21]:
correct = 0
total = 0
for i, data in enumerate(testloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = resnet_model(inputs)
        if(i==0):
          print(f"Output dimensions: {outputs.size()}\nLabel dimensions: {labels.size()}")
        outputs = torch.argmax(outputs, dim=1)
        correct += torch.sum(torch.eq(outputs, labels))
        total += 64
print(f"Test accuracy: {correct/total*100:.2f}%")


Output dimensions: torch.Size([64, 1000])
Label dimensions: torch.Size([64])
63.26%
